In [6]:
!pip install dtw-python

In [1]:
import tensorflow as tf
import numpy as np
from dtw import *

def euclideanDistance(x, y):
    dist = tf.cast(tf.norm(x-y), tf.float32)
    #dist = tf.cast(tf.math.sqrt(tf.math.reduce_sum(tf.math.squared_difference(x,y))), tf.float32)
    return dist

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [204]:
# Classique convolution
def convolution_1D(inputs, weights):
    output_final = tf.TensorArray(dtype=tf.float32, size=(inputs.shape[-2] - weights.shape[0] + 1))
    for j in range(0, inputs.shape[-2] - weights.shape[0] + 1):
        output_final = output_final.write(
            j,
            tf.math.reduce_sum(tf.linalg.matmul(inputs[j:j + weights.shape[0]], weights),axis=[0,1])
        )
    output_final = output_final.stack()
    
    return output_final


class CNN1D(tf.keras.layers.Conv1D):
    def call(self, inputs):
        output = tf.map_fn(lambda inp: convolution_1D(inp, self.kernel) + self.bias, inputs)
        return tf.nn.relu(output)


## Fonction DTW Tensor

In [288]:
#Fonction qui crée la matrice DTW
@tf.function
def DTW_TF(S, S1, d=euclideanDistance):
    cost_matrix = []
    cost_matrix.append(tf.cast(tf.stack([0, *([np.inf] * S1.shape[0])]), "float32"))
    for i in range(1,S.shape[0]+1):
        sub_cost_j = [np.inf]
        for j in range(1, S1.shape[0]+1):
            dst = d(S[i-1], S1[j-1])
            mat_dt =[
            dst + sub_cost_j[j-1],
            dst + cost_matrix[i-1][j-1],
            dst + cost_matrix[i-1][j]
            ]
            sub_cost_j.append(tf.reduce_min(mat_dt))
        cost_matrix.append(tf.stack(sub_cost_j))
    return DTW_minimal_path(tf.stack(cost_matrix))


# Renvoie le chemin dtw optimal en 2 fonctions
@tf.function
def loop_function(i, j, best_path, compteur, cost_mat):
    compteur += 1
    cost_min = tf.stack([
        cost_mat[i-1, j-1],
        cost_mat[i, j-1],
        cost_mat[i-1, j]
    ])
    n_min = tf.math.argmin(cost_min)
    i, j = tf.case([
        (tf.equal(n_min,0), lambda: (i-1, j-1)),
        (tf.equal(n_min,1), lambda: (i, j-1)),
        (tf.equal(n_min,2), lambda: (i-1, j))
                    ])
    best_path = best_path.write(compteur, (i-1, j-1))
    return i, j, best_path, compteur, cost_mat

@tf.function
def DTW_minimal_path(cost_mat):
    i = cost_mat.shape[0] - 1
    j = cost_mat.shape[1] - 1
    compteur = 0
    best_path = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    best_path = best_path.write(compteur, (i-1, j-1))
    cond = lambda i, j, best_path, compteur, cost_mat: tf.logical_and(tf.greater(i, 0), tf.greater(j, 0))
    i, j, best_path, compteur, cost_mat = tf.while_loop(cond, loop_function, [i, j, best_path, compteur, cost_mat])
    best_path = tf.cast(best_path.stack()[:-1][::-1], tf.int64)
    
    mat_allign = tf.sparse.SparseTensor(indices=best_path,
                                       values=tf.ones(tf.shape(best_path)[0], dtype=tf.dtypes.float32), 
                                       dense_shape=[cost_mat.shape[0] - 1, cost_mat.shape[1] - 1])
    return mat_allign

@tf.function
def slice_alignment(slice_input, weights, minmax='min'):
    output_list = tf.TensorArray(dtype=tf.float32, size=(weights.shape[0]))
    for filt in range(weights.shape[0]):
        # recuperation des "meilleurs chemins"
        t_weight = weights[filt]
        # Iwana DWA
        mat_allign = tf.sparse.to_dense(DTW_TF(slice_input, t_weight))
        # réalignement
        mat_allign = tf.reshape(mat_allign, (slice_input.shape[0], t_weight.shape[0]))
        output_list = output_list.write(filt, mat_allign)
    w_allign = output_list.stack() @ weights
    output = tf.math.reduce_sum(slice_input*weights, axis=[2,1])
    return output


# Fonction de base pour DWA de iwana
@tf.function
def conv1D_weight_alignment(inputs, weights):
    weights = tf.transpose(weights, perm=[2, 0, 1])
    output_final = tf.TensorArray(dtype=tf.float32, size=(inputs.shape[-2] - weights.shape[-2] + 1))
    for j in range(0, inputs.shape[-2] - weights.shape[-2] + 1):
        res = slice_alignment(tf.slice(inputs, (j, 0), (weights.shape[-2:])), weights)
        output_final = output_final.write(j, res)
    return output_final.stack()

class DWA_CNN(CNN1D):
    def call(self, inputs):
        output = tf.map_fn(lambda inp: conv1D_weight_alignment(inp, self.kernel) + self.bias, inputs)
        return tf.nn.relu(output)



## Fonction DTW numpy

In [287]:
def dtw_path(s1, s2):
    if s1.shape[0] == 1:
        return np.ones([1,1]).astype("float32")
    dtw_f = dtw(s1,s2, step_pattern="symmetric1")
    mat_allign = np.zeros((s1.shape[0], s2.shape[0]))
    for ind in zip(dtw_f.index1, dtw_f.index2):
        mat_allign[ind] = 1
    return mat_allign.astype("float32")

@tf.function
def tf_function(t_input, t_weight):
    T = tf.numpy_function(dtw_path, (t_input, t_weight), [tf.dtypes.float32])
    return T

@tf.function
def slice_alignment_np(slice_input, weights):
    output_list = tf.TensorArray(dtype=tf.dtypes.float32, size=(weights.shape[0]))
    for filt in range(weights.shape[0]):
        t_weight = weights[filt]
        mat_allign = tf_function(slice_input, t_weight)
        mat_allign = tf.reshape(mat_allign, (slice_input.shape[0], t_weight.shape[0]))
        output_list = output_list.write(filt, mat_allign)
    w_allign = output_list.stack() @ weights
    output = tf.math.reduce_sum(slice_input*weights, axis=[2,1])
    return output

@tf.function
def conv1D_weight_alignment_np(inputs, weights):
    weights = tf.transpose(weights, perm=[2, 0, 1])
    output_final = tf.TensorArray(dtype=tf.dtypes.float32, size=(inputs.shape[-2] - weights.shape[-2] + 1))
    for j in range(0, inputs.shape[-2] - weights.shape[-2] + 1):
        res = slice_alignment_np(tf.slice(inputs, (j, 0), (weights.shape[-2:])), weights)
        output_final = output_final.write(j, res)
    return output_final.stack()

class DWA_CNN_np(CNN1D):
    def call(self, inputs):
        output = tf.map_fn(lambda inp: conv1D_weight_alignment_np(inp, self.kernel) + self.bias, inputs)
        return tf.nn.relu(output)


# Tensor conv1d 

In [5]:
class Conv1D_DTW(tf.keras.layers.Conv1D):
    def call(self, inputs):
        mat_allign = tf.reshape(tf_function(inputs, self.kernel), tf.shape(kernel))
        input_allign = tf.linalg.matmul(mat_allign, self.kernel)
        result = self.convolution_op(
             inputs, input_allign
        )
        if self.use_bias:
            result = result + self.bias
        return result


 ## Test

In [278]:
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Flatten, Conv1D
from tensorflow.keras.utils import to_categorical


randi = np.random.random((300, 50, 2))
y_train = np.random.randint(1, 3, 300)
y_train = to_categorical(y_train)

In [289]:
%%time

tf.random.set_seed(1234)
model = Sequential([
    InputLayer(randi.shape[1:]),
    DWA_CNN_np(5, 2),
    Flatten(),
    Dense(3, activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy', metrics='accuracy')


model.fit(randi, y_train, epochs=3)

Model: "sequential_102"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dwa_cnn_np_65 (DWA_CNN_np)  (None, 49, 5)             25        
                                                                 
 flatten_102 (Flatten)       (None, 245)               0         
                                                                 
 dense_102 (Dense)           (None, 3)                 738       
                                                                 
Total params: 763
Trainable params: 763
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
10/10 [==============================] - 43s 3s/step - loss: 1.0265 - accuracy: 0.5167
Epoch 2/3
10/10 [==============================] - 29s 3s/step - loss: 0.8200 - accuracy: 0.4867
Epoch 3/3
10/10 [==============================] - 28s 3s/step - loss: 0.7652 - accuracy: 0.4833
CPU times: total: 2min
Wall

In [290]:
%%time

tf.random.set_seed(1234)
model_tensor = Sequential([
    InputLayer(randi.shape[1:]),
    DWA_CNN(5, 2),
    Flatten(),
    Dense(3, activation='softmax')
])

model_tensor.summary()
model_tensor.compile(loss='categorical_crossentropy', metrics='accuracy')

model_tensor.fit(randi, y_train, epochs=3)

Model: "sequential_103"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dwa_cnn_12 (DWA_CNN)        (None, 49, 5)             25        
                                                                 
 flatten_103 (Flatten)       (None, 245)               0         
                                                                 
 dense_103 (Dense)           (None, 3)                 738       
                                                                 
Total params: 763
Trainable params: 763
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
10/10 [==============================] - 127s 4s/step - loss: 1.0265 - accuracy: 0.5167
Epoch 2/3
10/10 [==============================] - 32s 3s/step - loss: 0.8200 - accuracy: 0.4867
Epoch 3/3
10/10 [==============================] - 28s 3s/step - loss: 0.7652 - accuracy: 0.4833
CPU times: total: 5min 37s

In [254]:
%%time

tf.random.set_seed(1234)
model_tensor = Sequential([
    InputLayer(randi.shape[1:]),
    DWA_CNN_test(5, 2),
    Flatten(),
    Dense(3, activation='softmax')
])

model_tensor.summary()
model_tensor.compile(loss='categorical_crossentropy', metrics='accuracy')

model_tensor.fit(randi, y_train, epochs=3)

NameError: name 'DWA_CNN_test' is not defined

(1, 1)

In [ ]:
# Classique convolution
def convolution_1D(inputs, weights):
    inputs = tf.reshape(tf.squeeze(inputs))
    output_list = []
    output_list.append(tf.math.reduce_sum(tf.linalg.matmul(inputs[j:j + weights.shape[0]], weights), axis=[0,1]))
    output_final = tf.stack(output_list)
    return output_final


class CNN1D(tf.keras.layers.Layer):
    def __init__(self, n_filters=8, kernel_size=3):
        super(CNN1D, self).__init__()
        self.n_filters = n_filters
        self.kernel_size = kernel_size
        self.b = self.add_weight(shape=(n_filters,), initializer="zeros", trainable=True)

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(self.kernel_size, int(input_shape[-1]), self.n_filters),
            initializer="glorot_uniform", trainable=True
        )

    def call(self, inputs):
        output = tf.map_fn(lambda inp: convolution_1D(inp, self.w) + self.b,
                           tf.image.extract_patches(images=inputs,
                           sizes=[tf.shape(inputs)[0], 1, self.kernel_size, 1],
                           strides=[1, 1, 1, 1],
                           rates=[1, 1, 1, 1],
                           padding='VALID')
)
        return tf.nn.relu(output)

In [291]:
%%time
tf.random.set_seed(1234)
model_tensor = Sequential([
    InputLayer(randi.shape[1:]),
    CNN1D(5, 2),
    Flatten(),
    Dense(3, activation='softmax')
])

model_tensor.summary()
model_tensor.compile(loss='categorical_crossentropy', metrics='accuracy')

model_tensor.fit(randi, y_train, epochs=3)

Model: "sequential_104"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cnn1d_17 (CNN1D)            (None, 49, 5)             25        
                                                                 
 flatten_104 (Flatten)       (None, 245)               0         
                                                                 
 dense_104 (Dense)           (None, 3)                 738       
                                                                 
Total params: 763
Trainable params: 763
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
10/10 [==============================] - 15s 48ms/step - loss: 1.1097 - accuracy: 0.4767
Epoch 2/3
10/10 [==============================] - 0s 39ms/step - loss: 0.8039 - accuracy: 0.4867
Epoch 3/3
10/10 [==============================] - 0s 41ms/step - loss: 0.7707 - accuracy: 0.4733
CPU times: total: 14.3 

In [292]:
%%time
tf.random.set_seed(1234)
model_tensor = Sequential([
    InputLayer(randi.shape[1:]),
    Conv1D(5, 2, activation='relu'),
    Flatten(),
    Dense(3, activation='softmax')
])

model_tensor.summary()
model_tensor.compile(loss='categorical_crossentropy', metrics='accuracy')

model_tensor.fit(randi, y_train, epochs=3)

Model: "sequential_105"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 49, 5)             25        
                                                                 
 flatten_105 (Flatten)       (None, 245)               0         
                                                                 
 dense_105 (Dense)           (None, 3)                 738       
                                                                 
Total params: 763
Trainable params: 763
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
10/10 [==============================] - 1s 3ms/step - loss: 1.0265 - accuracy: 0.5167
Epoch 2/3
10/10 [==============================] - 0s 2ms/step - loss: 0.8200 - accuracy: 0.4867
Epoch 3/3
10/10 [==============================] - 0s 4ms/step - loss: 0.7652 - accuracy: 0.4833
CPU times: total: 766 ms
Wa

In [ ]:
S = [1, 3, 3, 3, 2, 0, 1]
S1 = [0, 1, 3, 2, 2, 0, 1]
S = randi[1]
S1 = randi[2]

# distance = dtw.distance(S, S1)
# print(distance)
print(dtw(S,S1, step_pattern="symmetric1").index1)

S = tf.convert_to_tensor(S)
S1 = tf.convert_to_tensor(S1)
print(tf.sparse.to_dense(DTW_TF(S, S1)))
print(dtw_path(S, S1))

[ 0  1  2  3  4  4  5  6  7  8  9 10 11]
tf.Tensor(
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]], shape=(12, 12), dtype=float32)
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0